<a href="https://colab.research.google.com/github/liberty0081/DeepLearningColab/blob/develop/NeuralNetworkLayer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

class Softmax2CrossEntropy():
    def __init__(self):
        self.y    = None
        self.t    = None
        self.flag = False

    def forward(self, x, t):
        self.y = np.empty(x.shape)
        self.t = t
        
        for i in range(self.y.shape[0]):
            c = np.amax(x[i])
            self.y[i] = np.exp(x[i] - c)/(np.sum(np.exp(x[i] - c)))

        self.flag = True
        
        return -(np.sum(self.t*np.log(self.y)))/self.t.shape[0]

    def test_forward(self, x):
        y = np.empty(x.shape)

        for i in range(y.shape[0]):
            c = np.amax(x[i])
            y[i] = np.exp(x[i] - c)/(np.sum(np.exp(x[i] - c)))
        
        return y

    def test_loss(self, x, t):
        y = self.test_forward(x)
        return -(np.sum(t*np.log(y)))/t.shape[0]

    def backward(self):
        if self.flag:
            self.flag = False
            return (self.y - self.t)/self.t.shape[0]
        
        else:
            raise Exception("you have to run forward method before")
            
class Swish():
    def __init__(self):
        self.x    = None
        self.flag = False

    def forward(self, x):
        self.x    = x
        self.flag = True
        return self.x/(1 + np.exp(-self.x))

    def test_forward(self, x):
        return x/(1 + np.exp(-x))

    def backward(self, delta):
        if self.flag:
            self.flag = False
            return delta*((1 + np.exp(-self.x)*(1 + self.x))/((1 + np.exp(-self.x))**2))
        
        else:
            raise Exception("you have to run forward method before")

class ReLU():
    def __init__(self):
        self.x    = None
        self.flag = False

    def forward(self, x):
        self.x    = x
        self.flag = True
        return np.maximum(0, self.x)

    def test_forward(self, x):
        return np.maximum(0, x)

    def backward(self, delta):
        if self.flag:
            self.flag = False
            return delta*((self.x > 0).astype(np.int))
        
        else:
            raise Exception("you have to run forward method before")

class Sigmoid():
    def __init__(self):
        self.x    = None
        self.flag = False

    def forward(self, x):
        self.x    = x
        self.flag = True
        return 1/(1 + np.exp(-self.x))

    def test_forward(self, x):
        return 1/(1 + np.exp(-x))

    def backward(self, delta):
        if self.flag:
            self.flag = False
            return delta*((np.exp(-self.x))/(1 + np.exp(-self.x))**2)
        
        else:
            raise Exception("you have to run forward method before")

class Affine():
    def __init__(self, inputSize, outputSize):
        self.data  = {"IN":inputSize, "OUT":outputSize}
        self.W     = np.random.randn(self.data["IN"], self.data["OUT"])*0.1
        self.b     = np.random.randn(self.data["OUT"])*0.1
        self.dW    = np.zeros_like(self.W)
        self.db    = np.zeros_like(self.b)
        self.delta = None
        self.x     = None
        self.flag  = False 


    def forward(self, x):
        self.x    = x
        self.flag = True

        return np.dot(self.x, self.W) + self.b

    def test_forward(self, x):
        return np.dot(x, self.W) + self.b

    def backward(self, delta):
        if self.flag:
            self.flag = False
            self.delta = delta
            self.dW = np.dot(self.x.T, self.delta)
            self.db = np.sum(self.delta, axis = 0)

            return np.dot(self.delta, self.W.T)
        
        else:
            raise Exception("you have to run forward method before")

class BatchNorm():
    def __init__(self):
        self.W      = None
        self.b      = None
        self.mu     = None
        self.sigma  = None
        self.m      = None
        self.dW     = None
        self.db     = None
        self.x      = None
        self.x_hat  = None
        self.delta  = None
        self.flag   = False

    def forward(self, x):
        self.x    = x
        self.m    = self.x.shape[0]
        self.flag = True

        if self.W is None:
            self.W = np.ones_like(self.x)
        
        if self.b is None:
            self.b = np.zeros_like(self.x)

        self.mu    = np.sum(self.x, axis = 0)/self.x.shape[0]
        self.sigma = np.sum((self.x - self.mu)**2, axis = 0)/self.x.shape[0]
        self.x_hat = ((self.x - self.mu)/np.sqrt(self.sigma + 1e-7))

        return self.W*self.x_hat + self.b

    def test_forward(self, x):
        m     = x.shape[0]
        mu    = np.sum(x, axis = 0)/x.shape[0]
        sigma = np.sum((x - mu)**2, axis = 0)/x.shape[0]
        x_hat = ((x - mu)/np.sqrt(sigma + 1e-7))

        if self.W is None or self.b is None:
            return 1*x_hat

        else:
            return self.W*x_hat + self.b


    def backward(self, delta):
        if self.flag:
            self.flag = False

            self.db   = delta
            self.dW   = delta*self.x_hat
        
            a         = ((self.x - self.mu) - np.sum(self.x - self.mu, axis = 0)/self.m)*2/self.m

            row_size1    = a.shape[0]
            column_size1 = a.shape[1]

            b = a.T.flatten()
            dsigma = np.zeros(column_size1*row_size1*column_size1)

            indx1  = (np.arange(a.size)*column_size1 + np.floor(np.arange(a.size)/row_size1)).astype(int)
            dsigma[indx1] = b
            dsigma = dsigma.reshape(column_size1, row_size1, column_size1)

            dx_hat =  ((np.resize(np.identity(self.m), (self.sigma.size, self.m, self.m)) - 1/self.m)*((self.W*(self.sigma + 1e-7)).T)[:, np.newaxis, np.newaxis]) \
            - np.dot(dsigma, (self.W*(self.x - self.mu)/2).T)/(np.sqrt((self.sigma + 1e-7)[:, np.newaxis, np.newaxis])*(self.sigma + 1e-7)[:, np.newaxis, np.newaxis])

            c = np.dot(dx_hat, delta)
            row_size2    = c.shape[1]
            column_size2 = c.shape[2]
            indx2 = (np.arange(row_size2*column_size2)*column_size2 + np.floor(np.arange(row_size2*column_size2)/row_size2)).astype(int)

            return c.flatten()[indx2].reshape(column_size2, row_size2)

        else:
            raise Exception("you have to run forward method before")